In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

### K-Means Cluster Analysis of Profitability

In [2]:
financial_df = pd.read_csv('./data/Financial Characteristics (ACSST5Y2022.S2503)/ACSST5Y2022.S2503-Data.csv')
household_df = pd.read_csv('./data/Financial Characteristics (ACSST5Y2022.S2503)/ACSST5Y2022.S2503-Data.csv')
shortage_tracts_gdf = gpd.read_file('./data/shortage_tracts_gdf_1pcp220.geojson')

financial_df = financial_df.iloc[1:, :]
household_df = household_df.iloc[1:, :]

# Filter unserved columns and Add unserved density columns
shortage_tracts_gdf = shortage_tracts_gdf[(shortage_tracts_gdf['unserved_medicaid'] > 0) | (shortage_tracts_gdf['unserved_commercial'] > 0)]
shortage_tracts_gdf['unserved_medicaid/km2'] = (shortage_tracts_gdf['unserved_medicaid'] / shortage_tracts_gdf['ALAND'] * 1000000).round(1)
shortage_tracts_gdf['unserved_commercial/km2'] = (shortage_tracts_gdf['unserved_commercial'] / shortage_tracts_gdf['ALAND'] * 1000000).round(1)

columns_to_keep = ['GEOID', 'geometry', 'unserved_medicaid', 'unserved_medicaid/km2', 'unserved_commercial', 'unserved_commercial/km2']
shortage_tracts_gdf = shortage_tracts_gdf[columns_to_keep]

C:\Users\kl3539\AppData\Local\Temp\ipykernel_18804\1566372108.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,278,279,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449

In [22]:
income_df = financial_df[['GEO_ID', 'S2503_C01_013E', 'S2503_C01_024E']]
income_df.rename(columns={'S2503_C01_013E': 'Median household income', 'S2503_C01_024E': 'Median monthly housing cost'}, inplace=True)
income_df['Median household income'] = pd.to_numeric(income_df['Median household income'], errors='coerce')
income_df['Median monthly housing cost'] = pd.to_numeric(income_df['Median monthly housing cost'], errors='coerce')
income_df = income_df.dropna()
income_df['Median household disposable income'] = income_df['Median household income'] - income_df['Median monthly housing cost'] * 12

print(income_df)

                    GEO_ID  Median household income  \
1     1400000US36001000100                  44547.0   
2     1400000US36001000201                  33688.0   
3     1400000US36001000202                  32585.0   
4     1400000US36001000301                  43214.0   
5     1400000US36001000302                  50875.0   
...                    ...                      ...   
5407  1400000US36123150301                  71713.0   
5408  1400000US36123150302                  47122.0   
5409  1400000US36123150400                  81038.0   
5410  1400000US36123150501                  61123.0   
5411  1400000US36123150502                  70179.0   

      Median monthly housing cost  Median household disposable income  
1                           985.0                             32727.0  
2                           983.0                             21892.0  
3                           674.0                             24497.0  
4                          1121.0                  

C:\Users\kl3539\AppData\Local\Temp\ipykernel_18804\2675522388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income_df.rename(columns={'S2503_C01_013E': 'Median household income', 'S2503_C01_024E': 'Median monthly housing cost'}, inplace=True)
C:\Users\kl3539\AppData\Local\Temp\ipykernel_18804\2675522388.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income_df['Median household income'] = pd.to_numeric(income_df['Median household income'], errors='coerce')
C:\Users\kl3539\AppData\Local\Temp\ipykernel_18804\2675522388.py:4: SettingWithCopyWarning: 
A val